In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import utils
import tensorflow as tf
import json
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model
from utils.extractors import Pathways
from utils import extractors, transformers, modelers
from utils.extractors import Pathways

In [ ]:
images = Pathways.img_path.glob('*.jpg')

In [ ]:
image_fn = next(images)
file_id = image_fn.stem
label_fn = Pathways.labels_path / f'{file_id}.json'

image = extractors.load_image(str(image_fn)).numpy()

with open(label_fn, 'r', encoding = "utf-8") as f:
    label = json.load(f)

In [ ]:
shapes = label['shapes']
shape = shapes[0]
points, name = shape['points'], shape['label']
(x1, y1), (x2, y2) = [tuple(int(coord) for coord in point) for point in points]

face = image[y1:y2, x1:x2]
face_resized = cv2.resize(face, (224, 224))

In [ ]:
fig, axes = plt.subplots(figsize=(10,10), ncols=2)

cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)

ax = axes[0]
ax.imshow(image)
title = f'Coords: ({x1}, {y1}), ({x2}, {y2})'
ax.set_title(title)

ax = axes[1]
ax.imshow(face_resized)
ax.set_title(name)

plt.show()

# Tensorflow Dataset

In [ ]:
val = transformers.create_ds('val')
val_iter = val.as_numpy_iterator()

In [ ]:
X, (y_classes, y_coords) = next(val_iter)
X.shape, y_coords.shape

## Coord Swap Details

> Current Coords
>> [x1, y1, x2, y2]

> `crop_and_resize` Coords
>>  [y1, x1, y2, x2]

In [ ]:
[i+4 for i in col_swap]

In [ ]:
col_swap = [1, 0, 3, 2]
y_coords_brendan, y_coords_kara = y_coords[:, [1, 0, 3, 2]], y_coords[:, [i+4 for i in col_swap]]
X_cropped_brendan = tf.image.crop_and_resize(X, y_coords_brendan, np.array(range(X.shape[0])),  crop_size=(120, 120))
X_cropped_kara = tf.image.crop_and_resize(X, y_coords_kara, np.array(range(X.shape[0])),  crop_size=(120, 120))

In [ ]:
i += 1
image, face_resized_brendan, face_resized_kara = X[i], X_cropped_brendan[i].numpy(), X_cropped_kara[i].numpy()
coords_brendan, coords_kara = y_coords_brendan[i], y_coords_kara[i]

fig, axes = plt.subplots(figsize=(10,10), ncols=3)

# cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)

ax = axes[0]
ax.imshow(image)
# title = f'Coords: ({x1}, {y1}), ({x2}, {y2})'
# ax.set_title(title)

ax = axes[1]
ax.imshow(face_resized_brendan)
# ax.set_title(name)

ax = axes[2]
ax.imshow(face_resized_kara)

plt.show()